## QNexus Client Demonstration

A short demonstration of the Nexus programming environment and our upcoming qnexus client.

In [3]:
import qnexus as qnx

### Projects

Projects allow us to group remote resources together to be organized and shared with others.

In [2]:
ghz_project = qnx.project.create(
    name="GHZ state quickstart",
    description="A Demonstration of Nexus features",
)

### Pandas integration

We provide a `.df()` method on many constructs to display them in a pandas data frame.

In [3]:
ghz_project.df()

,name,description,created,modified,contents_modified,id
0,GHZ state quickstart,A Demonstration of Nexus features,2024-05-27 05:46:05.941473+00:00,2024-05-27 05:46:05.941473+00:00,2024-05-27 05:46:05.946637+00:00,1d13e2fa-3e41-4cd1-b271-c76743bf1836


### Contexts

When working in a project, we can set it as the active context.

In [4]:
qnx.context.set_active_project(ghz_project)

<Token var=<ContextVar name='qnexus_project' default=None at 0x7f54fe5ddee0> at 0x7f54fbd4cdc0>

### Pytket

The pytket programming environment is included and well supported in Nexus.

Here we use it to build a circuit that prepares a GHZ state.

In [2]:
from pytket import Circuit

ghz_circuit = Circuit(3, 3).H(0).CX(0,1).CX(1,2)

In [6]:
from pytket.circuit.display import render_circuit_jupyter as draw

draw(ghz_circuit)

### Querying and uploading

We provide many utilities for working with cloud based resources without handles.

By giving names to resources we can avoid duplication and restart notebooks easily.

In [7]:
ghz_ref = qnx.circuit.upload(
    circuit=ghz_circuit,
    name="ghz_circuit",
    description="A prepared GHZ state",
)

In [8]:
ghz_ref.df()

,name,description,created,modified,project,id
0,ghz_circuit,A prepared GHZ state,2024-05-27 05:49:01.184334+00:00,2024-05-27 05:49:01.184334+00:00,GHZ state quickstart,c58a5013-77f5-4067-943d-cd0d8963f28e


### State Vector Simulator

As a sanity check, let's compile and run our circuit on a state vector simulator.

In [9]:
state_vec_target = qnx.QulacsConfig()

In [10]:
ghz_compile_job_ref = qnx.start_compile_job(
    circuits=[ghz_ref],
    name="compile_ghz",
    backend_config=state_vec_target,
)

In [18]:
ghz_compile_job_ref

CompileJobRef(id=UUID('4aec06cd-cdb2-4de8-95c2-69b11b8cbbd2'), annotations=Annotations(name='compile_ghz', description=None, properties=OrderedDict(), created=None, modified=None), job_type=<JobType.COMPILE: 'COMPILE'>, last_status=<StatusEnum.SUBMITTED: 'Circuit has been submitted.'>, last_message='', project=ProjectRef(id=UUID('1d13e2fa-3e41-4cd1-b271-c76743bf1836'), annotations=Annotations(name='GHZ state quickstart', description='A Demonstration of Nexus features', properties=OrderedDict(), created=datetime.datetime(2024, 5, 27, 5, 46, 5, 941473, tzinfo=TzInfo(UTC)), modified=datetime.datetime(2024, 5, 27, 5, 46, 5, 941473, tzinfo=TzInfo(UTC))), contents_modified=datetime.datetime(2024, 5, 27, 5, 46, 5, 946637, tzinfo=TzInfo(UTC))))

In [11]:
qnx.job.wait_for(ghz_compile_job_ref)

JobStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='The job is completed.', error_detail=None, completed_time=datetime.datetime(2024, 5, 27, 5, 50, 12, 831141, tzinfo=datetime.timezone.utc), queued_time=None, submitted_time=datetime.datetime(2024, 5, 27, 5, 50, 4, 629322, tzinfo=datetime.timezone.utc), running_time=datetime.datetime(2024, 5, 27, 5, 50, 4, 682531, tzinfo=datetime.timezone.utc), cancelled_time=None, error_time=None, queue_position=None)

In [12]:
qnx.job.get().df()

,name,description,created,modified,job_type,last_status,project,id
0,compile_ghz,None,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,JobType.COMPILE,StatusEnum.COMPLETED,unimplemented,4aec06cd-cdb2-4de8-95c2-69b11b8cbbd2


In [13]:
qnx.job.results(ghz_compile_job_ref).df()

,name,description,created,modified,project,id
0,ghz_circuit-compilation,,2024-05-27 05:50:12.685242+00:00,2024-05-27 05:50:12.685242+00:00,GHZ state quickstart,9dbe1c13-c730-411c-97b4-de3158e68863


In [14]:
compile_job_results = qnx.job.results(ghz_compile_job_ref)

first_job_result = compile_job_results[0]

In [15]:
input_circuit_ref = first_job_result.get_input()

input_circuit = input_circuit_ref.download_circuit()

draw(input_circuit)

### Compilation output

We can examine the circuits in the web UI, or download them into our session.

In [16]:
compiled_circuit_ref = first_job_result.get_output()

compiled_circuit = compiled_circuit_ref.download_circuit()

draw(compiled_circuit)


### Execution

We can then execute a circuit on a target device without needing to re-upload it.

In [19]:
ghz_execute_job_ref = qnx.start_execute_job(
    circuits=[compiled_circuit_ref], # <- Don't have to download, can just use the reference.
    name="execute_ghz",
    backend_config=state_vec_target,
    n_shots=[None],
)

In [20]:
qnx.job.get().df()

,name,description,created,modified,job_type,last_status,project,id
0,compile_ghz,None,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,JobType.COMPILE,StatusEnum.COMPLETED,unimplemented,4aec06cd-cdb2-4de8-95c2-69b11b8cbbd2
1,execute_ghz,None,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,JobType.EXECUTE,StatusEnum.COMPLETED,unimplemented,8e4df623-0162-48b4-a6dc-9133aea8287f


In [21]:
qnx.job.wait_for(ghz_execute_job_ref)

JobStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='The job is completed.', error_detail=None, completed_time=datetime.datetime(2024, 5, 27, 5, 55, 19, 561858, tzinfo=datetime.timezone.utc), queued_time=None, submitted_time=datetime.datetime(2024, 5, 27, 5, 55, 16, 475116, tzinfo=datetime.timezone.utc), running_time=datetime.datetime(2024, 5, 27, 5, 55, 16, 720920, tzinfo=datetime.timezone.utc), cancelled_time=None, error_time=None, queue_position=None)

In [22]:
execute_job_result = qnx.job.results(ghz_execute_job_ref)

first_execution_result = execute_job_result[0]

backend_result = first_execution_result.download_result()

### Results

We can see that we do get our expected outcome of a superposition of `|000>` and `|111>`

In [23]:
backend_result.get_state()

array([0.70710678+6.77244997e-17j, 0.        +0.00000000e+00j,
       0.        +0.00000000e+00j, 0.        +0.00000000e+00j,
       0.        +0.00000000e+00j, 0.        +0.00000000e+00j,
       0.        +0.00000000e+00j, 0.70710678-4.32978028e-17j])

### Measurements

We should expect that if we perform a measurement in the X basis on the third qubit, that we will produce one of two bell states depending on the outcome.

In [24]:
ghz_measure_circuit = ghz_ref.download_circuit()

# Measurement in the X basis to register 0.
ghz_measure_circuit.H(2).Measure(2, 0)

# Measurements in the Z basis of the remaining qubits into registers 1 and 2.
ghz_measure_circuit.add_barrier([0,1,2]).Measure(0, 1).Measure(1, 2)

[H q[0]; CX q[0], q[1]; CX q[1], q[2]; H q[2]; Measure q[2] --> c[0]; Barrier q[0], q[1], q[2]; Measure q[0] --> c[1]; Measure q[1] --> c[2]; ]

In [25]:
draw(ghz_measure_circuit)

In [28]:
def compile_and_execute(circuit_ref: qnx.circuit.CircuitRef):
    """Compile and execute a circuit on the H1-Emulator."""

    counts_target = qnx.QuantinuumConfig(device_name="H1-Emulator")
    compiled_circuit= qnx.compile(
        circuits=[circuit_ref],
        name=f"compile_{circuit_ref.annotations.name}",
        backend_config=counts_target,
    )[0]

    result = qnx.execute(
        circuits=[compiled_circuit_ref],
        name=f"execute_{circuit_ref.annotations.name}",
        backend_config=counts_target,
        n_shots=[1024],
    )[0]

    return result

In [26]:
measure_ghz_ref = qnx.circuit.upload(
    circuit=ghz_measure_circuit,
    name="measure_ghz_circuit",
    description="A GHZ state with a measurement on the 3rd qubit then the other qubits",
)

In [29]:
backend_result = compile_and_execute(measure_ghz_ref)

### Results

We can see that we do get two bell states depending on the first measurement.

In [30]:
backend_result.get_counts()

Counter({(1, 0, 0): 285,
         (0, 0, 0): 249,
         (1, 1, 1): 244,
         (0, 1, 1): 243,
         (1, 0, 1): 2,
         (1, 1, 0): 1})